In [1]:
import numpy as np
import scipy as sp

from typing import List, Set, Tuple

Testing out a new way to index every cell in the multiple level arrays

Expect quadrants to be indexed as 0123 from bottom left, to bottom right, to top left, to top right
(Hence to the output of the matricies in pure number form are actually x down, y right)

# Index Matricies

Functions to get:
* Children
* Parent
* Nearest Neighbour
* Interaction List
For a given set of coordinates in a given level

Also generator function for a list of index matricies to test out on.

In [2]:
def create_index_matricies(max_level:int):
    max_level = 3

    return [np.arange(4**l).reshape((2**l, 2**l)).transpose() for l in range(max_level+1)]


def get_children(parent_coords:Tuple[int]) -> List[Tuple[int]]:
    x, y = parent_coords
    return [
        (x*2, y*2),
        (x*2+1, y*2),
        (x*2, y*2+1),
        (x*2+1, y*2+1)
    ]

def parent(child_coords:Tuple[int]) -> Tuple[int]:
    return (child_coords[0]//2, child_coords[1]//2)


def nearest_neighbours(coords:Tuple[int], level:int) -> Set[Tuple[int]]:
    if level == 0:
        return set()

    x,y = coords
    max_coord = 2**level - 1
    neighbours = set()

    # middle row x_n = x
    if y != 0:
        neighbours.add((x,y-1))
    if y != max_coord:
        neighbours.add((x,y+1))

    # left row x_n = x-1
    if x !=0:
        neighbours.add((x-1,y))
        if y != 0:
            neighbours.add((x-1,y-1))
        if y != max_coord:
            neighbours.add((x-1,y+1))

    # right row x_n = x+1
    if x != max_coord:
        neighbours.add((x+1,y))
        if y != 0:
            neighbours.add((x+1,y-1))
        if y != max_coord:
            neighbours.add((x+1,y+1))
    
    return neighbours

def interaction_list(cell_coords:Tuple[int], level:int) -> Set[Tuple[int]]:
    if level <= 1:
        return set()
    
    own_neighbours = nearest_neighbours(cell_coords, level)
    parent_neighbours = nearest_neighbours(parent(cell_coords), level-1)
    all_possible = set()
    for p_n in parent_neighbours:
        all_possible.update(get_children(p_n))

    return all_possible - own_neighbours

In [3]:
max_level = 3

index_matricies = create_index_matricies(max_level)

In [4]:
cell = (1,0)
print(index_matricies[2][cell])
print({index_matricies[2][interactor] for interactor in interaction_list(cell,2)})

# weirdness is to output as expected, ie in xy coordinates
# [matrix for matrix in level_matricies]
[np.flip(np.transpose(matrix), 0) for matrix in index_matricies]

1
{3, 7, 8, 9, 10, 11, 12, 13, 14, 15}


[array([[0]]),
 array([[2, 3],
        [0, 1]]),
 array([[12, 13, 14, 15],
        [ 8,  9, 10, 11],
        [ 4,  5,  6,  7],
        [ 0,  1,  2,  3]]),
 array([[56, 57, 58, 59, 60, 61, 62, 63],
        [48, 49, 50, 51, 52, 53, 54, 55],
        [40, 41, 42, 43, 44, 45, 46, 47],
        [32, 33, 34, 35, 36, 37, 38, 39],
        [24, 25, 26, 27, 28, 29, 30, 31],
        [16, 17, 18, 19, 20, 21, 22, 23],
        [ 8,  9, 10, 11, 12, 13, 14, 15],
        [ 0,  1,  2,  3,  4,  5,  6,  7]])]

# Actual FMM

Create list of all the matricies of the various multipole and local expansions
These will be 3D matricies with dimensions of $(2^l, 2^l, 2p)$ where $l$ is the level and $p$ is the global precision for each expansion

In [5]:
from fmm import Particle

In [6]:
def create_expansion_matricies(max_level:int, precision:int):
    """Returns a list of matricies for the expansion coefficients to be placed into
    
    For each 'cell' the first value also stores the complex centre of that cell
    """

    expansion_matricies = [np.zeros((2**l,2**l,2*precision+1), dtype=complex) for l in range(max_level+1)]
    for l, matrix in enumerate(expansion_matricies):
        first_val = 1/(2**(l+1))
        # stop of 1 as this is the so called max value, but will never appear
        vals = np.arange(first_val,1,2*first_val)
        X,Y = np.meshgrid(vals, vals, indexing='ij')
        matrix[:,:,0] = X + 1j*Y
    
    return expansion_matricies

def get_particle_cell(particle:Particle, level:int) -> Tuple[int]:
    return (
        int(particle.centre.real * 2**level),
        int(particle.centre.imag * 2**level)
    )

In [12]:
def calculate_multipole(particle:Particle, precision:int, level:int, matrix) -> None:
    """Update the relevant cell in given level with multipole due to particle
    
    Parameters
    ----------
    particle : Particle
        particle whose effect to add
    precision : int
        precision in multipole expansion
    level : int
        the level at which to add the particle's effect
    matrix : NDArray
        the full matrix for the relevant level
    """

    cell = get_particle_cell(particle, level)
    matrix[cell][1] = particle.charge
    k = np.arange(1,precision,1)
    matrix[cell][2:precision+1] += -particle.charge * (particle.centre - matrix[cell[0],cell[1],0])**k / k

def cell_M2M(cell:Tuple[int], child:Tuple[int], precision:int, matrix, children_matrix):
    """M2M from children for a single cell"""

    child_multipole = children_matrix[child][1:precision+1]

    matrix[cell][1] += child_multipole[0]

    z0 = children_matrix[child][0] - matrix[cell][0]

    for l in range(1,precision):
        k = np.arange(1,l+1)
        matrix[cell][l+1] += -(child_multipole[0] * z0**l / l) \
            + np.sum(child_multipole[1:l+1] \
                        * z0**(l-k) * sp.special.binom(l-1,k-1))

def level_M2M(precision:int, level:int, matrix, children_matrix) -> None:
    """Perform M2M on a given level due to the multipoles in the child level"""
    # Can definitely optimise the way this is done
    for x in range(2**level):
        for y in range(2**level):
            for child in get_children((x,y)):
                cell_M2M((x,y),child,precision,matrix,children_matrix)

def cell_M2L(cell:Tuple[int], interactor:Tuple[int], precision:int, matrix) -> None:
    """Local expansion of a cell due to interactor"""

    z0 = matrix[interactor][0] - matrix[cell][0] # local expansion 'about origin' (so about cell)

    minus_and_plus = np.empty(precision-1)
    minus_and_plus[::2] = -1
    minus_and_plus[1::2] = 1

    k_vals = np.arange(1, precision)
    l_vals = np.arange(1, precision)

    interactor_multipole = matrix[interactor][1:precision+1]

    minus_bk_over_z0k = minus_and_plus * interactor_multipole[1:] / z0**k_vals

    matrix[cell][precision+1] += interactor_multipole[0] * np.log(-z0) + np.sum(minus_bk_over_z0k)
    matrix[cell][precision+2:] += interactor_multipole[0] / (l_vals * z0**l_vals) \
                    + (1/z0**l_vals) * np.sum(minus_bk_over_z0k * sp.special.binom(l_vals + k_vals - 1, k_vals-1))
    
def level_M2L(precision:int, level:int, matrix) -> None:
    """Do M2L for a given level"""

    for x in range(2**level):
        for y in range(2**level):
            for interactor in interaction_list((x,y),level):
                cell_M2L((x,y),interactor,precision,matrix)

def cell_L2L(cell:Tuple[int], child:Tuple[int], precision:int, matrix, child_matrix) -> None:
    """Distribute local expansion to child cells"""
    
    z0 = child_matrix[child][0] - matrix[cell][0]

    k_vals = np.arange(1,precision)
    
    cell_local = matrix[cell][precision+2:] # does not include l=0 of cell's local

    for l in range(precision):
        child_matrix[child][precision+1+l] += np.sum(cell_local * sp.special.binom(k_vals,l) * z0**(k_vals-l))

def level_L2L(precision:int, level:int, matrix, child_matrix) -> None:
    """Distribute all locals in a given level to children level"""

    for x in range(2**level):
        for y in range(2**level):
            for child in get_children((x,y)):
                cell_L2L((x,y), child, precision, matrix, child_matrix)


In [ ]:
def insert_particles(finest_matrix, max_level, particles:List[Particle], precision:int):

    for particle in particles:
        calculate_multipole(particle, precision, max_level, finest_matrix)

def upward_pass(precision:int, max_level:int, expansion_matricies):
    for level in range(max_level-1, -1, -1):
        level_M2M(precision, level, expansion_matricies[level], expansion_matricies[level+1])

def downward_pass(precision:int, max_level, expansion_matricies):
    """Perform downward pass"""

In [13]:
max_level = 1
precision = 4

expansion_matricies = create_expansion_matricies(max_level, precision)

particles = [
    Particle(1, 0.25*(1+1j)),
    Particle(2, 0.25*(3+1j)),
    Particle(-3, 0.25*(3+3j)),
    Particle(-2, 0.25*(1+3j))
]

for particle in particles:
    calculate_multipole(particle, precision, max_level, expansion_matricies[max_level])

level_M2M(precision, 0, expansion_matricies[0], expansion_matricies[1])

expansion_matricies[0][(0,0)][1:precision+1]

array([-2.+0.j        ,  0.+2.j        ,  0.+0.125j     ,  0.+0.08333333j])

In [45]:
expansion_matricies = create_expansion_matricies(3,4)
centres = [matrix[:,:,0] for matrix in expansion_matricies]

cell = (0,1)
expansion_matricies[1][cell[0], cell[1], 2:5]

[np.flip(np.transpose(matrix), 0) for matrix in centres]

array([0.+0.j, 0.+0.j, 0.+0.j])